In [ ]:
import os
import re
import random
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader

# Function to set random seed
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Function to prepare test data
def prepare_test_data(batch_size=64):
    ########## 필요한 코드 채워주세요 ##########
    transform = transforms.Compose([
        transforms.ToTensor(),  # 이미지를 PyTorch 텐서로 변환
        transforms.Normalize((0.5,), (0.5,))  # 픽셀 값을 -1과 1 사이로 정규화
    ])

    # FashionMNIST 테스트 데이터셋 다운로드 및 로드
    test_dataset = torchvision.datasets.FashionMNIST(
        root='./data',      # 데이터가 저장될 경로
        train=False,        # 테스트 데이터셋을 로드
        download=True,      # 해당 경로에 데이터가 없으면 다운로드
        transform=transform # 정의한 transform 적용
    )
    ###########################################
    return DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Function to calculate accuracy
def calculate_accuracy(logits, targets):
    pred = np.argmax(logits, axis=1)
    accuracy = 100. * np.sum(pred == targets.numpy()) / len(targets)
    return round(accuracy, 2)

# Main function to evaluate logits
def evaluate(logits_filename):
    set_seed()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    test_loader = prepare_test_data()
    print(f'Test dataset size: {len(test_loader.dataset)}')

    # dummy_logits = np.random.rand(10000, 10)
    # FashionMNIST 테스트셋 크기와 동일한 더미 logits 생성 (10000개, 클래스 10개)
    dummy_logits = np.random.rand(len(test_loader.dataset), 10)
    np.save("LimJunsung.npy", dummy_logits)

    try:
        # Ensure file name is valid
        assert re.match(r"^(r\d{8}|LimJunsung\d*)\.npy$", logits_filename), \
            "File name must start with 'jslim' followed by digits and end with '.npy'."

        # Load logits
        logits = np.load(logits_filename)

        #logits = np.load(logits_filename, allow_pickle=True)

        assert logits.shape == (len(test_loader.dataset), 10), \
            f"Logits shape mismatch: expected ({len(test_loader.dataset)}, 10)."

        # Calculate accuracy
        targets = torch.cat([target for _, target in test_loader]).cpu()
        accuracy = calculate_accuracy(logits, targets)

        print(f'{logits_filename[:-4]} - Accuracy: {accuracy:.2f}%')

    except AssertionError as e:
        accuracy = 0.0

if __name__ == "__main__":
    evaluate("LimJunsung.npy")


Using device: cpu
Test dataset size: 10000
LimJunsung - Accuracy: 10.00%
